In [3]:
import keras as K
import os.path

import sys

sys.path.append("../src")
from artstat import util
import numpy as np

Using TensorFlow backend.


In [4]:
import imp
imp.reload(util)

<module 'artstat.util' from '../src/artstat/util.py'>

In [5]:
datadir = "/home/pmilovanov/hg/my/data/gallery-pr/all"

path_train = os.path.join(datadir, "train")
path_test = os.path.join(datadir, "test")

glove = "/home/pmilovanov/data/glove/glove.840B.300d.txt"

In [159]:
words, vocab = util.load_vocab("../vocab.txt", 100000)

In [160]:
emb_matrix = util.load_embeddings(vocab, 300, glove)

In [161]:
emb_matrix.shape
len(vocab)

100000

In [183]:
imp.reload(util)

seqlen = 128
stride = 96
X, Y, Xu, Yu = util.load_data_sequences(path_train, vocab, seqlen, stride)
Yfake = np.ones((Yu.shape[0], 128), dtype="float32")


100%|##########| 22860/22860 [00:36<00:00, 626.31it/s]


In [201]:
input_x = K.Input((seqlen,), dtype="int32", name="input_x")
input_xu = K.Input((seqlen,), dtype="float32", name="input_xu")
input_y = K.Input((seqlen,), dtype="int32", name="input_y")
input_yu = K.Input((seqlen,), dtype="float32", name="input_yu")

reshaper_u  = K.layers.Reshape((seqlen,1))
resh_xu = reshaper_u(input_xu)
resh_yu = reshaper_u(input_yu)

dim = emb_matrix.shape[1] + 1

L_emb = K.layers.Embedding(emb_matrix.shape[0], 
                           emb_matrix.shape[1], 
                           input_length=seqlen, 
                          trainable=False,
                           weights=[emb_matrix],
                           name="embedding")


In [202]:
Lemb = L_emb
emb_x = Lemb(input_x)
emb_y = Lemb(input_y)

In [203]:
Lconcat = K.layers.Concatenate()
concat_x = Lconcat([emb_x, resh_xu])
concat_y = Lconcat([emb_y, resh_yu])

In [204]:
lstm =  K.layers.CuDNNLSTM(256, return_sequences=True)(concat_x)
lstm =  K.layers.CuDNNLSTM(256, return_sequences=True)(concat_x)
#lstm = K.layers.CuDNNLSTM(256, return_sequences=True)(lstm)
dense = K.layers.BatchNormalization()(lstm)
dense = K.layers.Dense(1024, activation="relu")(dense)
dense = K.layers.BatchNormalization()(dense)
dense = K.layers.Dense(1024, activation="relu")(dense)
dense = K.layers.BatchNormalization()(dense)
dense = K.layers.Dense(dim, activation="sigmoid")(dense)

dense.shape

TensorShape([Dimension(None), Dimension(128), Dimension(301)])

In [205]:
concat_y.shape

TensorShape([Dimension(None), Dimension(128), Dimension(301)])

In [206]:
#proximity = K.layers.dot([resh_x, resh_y], axes=-1, normalize=True)
import keras as K
from keras import backend as B

import tensorflow as tf


epsi = 0.0001

def realdot(a,b):
    d = K.layers.multiply([a, b])
    return B.sum(d, axis=-1)

def dotlayer(x):
    a,b = x
    
    p = realdot(a,b)
    
    an = B.sqrt(realdot(a,a) + B.epsilon())
    bn = B.sqrt(realdot(b,b) + B.epsilon())
    
    return tf.divide(p, K.layers.multiply([an, bn]) + B.epsilon())
    
def test2(x):
    a,b = x
    
    q = K.layers.subtract([a,b])
    q = B.sum(q, axis=-1)
#    q = B.mean(q, axis=-1, keepdims=True)
    return q

qA = B.variable(np.random.randn(6,10,12)) #np.array([[1,0], [3,4]]))
qB = B.variable(np.random.randn(6,10,12)) #np.array([[0,0], [-7,-8]]))
qq = dotlayer([qA, qB])
print(qq)
print(B.eval(qq))    
    
proximity = K.layers.Lambda(dotlayer)([concat_y, dense])
    
#proximity = K.layers.dot([concat_y, dense], axes=(2,2),normalize=True)




#proximity = K.layers.average(proximity, axis=-1)
proximity.shape

Tensor("truediv_12:0", shape=(6, 10), dtype=float32)
[[-0.05244785 -0.12994386  0.23185357 -0.13510628 -0.19214262 -0.15168397
   0.03622652 -0.5376563  -0.0483631   0.01413608]
 [-0.06501046 -0.12433026 -0.25961637 -0.18919732  0.5330739  -0.13260975
   0.17318347 -0.14005494  0.1490598   0.39558622]
 [ 0.25163382  0.01447667  0.3840088   0.04292454  0.42716476 -0.22890744
   0.32895193 -0.2226297  -0.29326904 -0.00343852]
 [-0.2746462   0.3062957  -0.1459034  -0.37260023  0.43964973 -0.21602046
   0.12064244  0.20292956 -0.330544   -0.12878366]
 [ 0.10982414 -0.03371133 -0.31320986  0.12506601 -0.01835176  0.41049019
  -0.05448675  0.08492289 -0.17325367 -0.19006237]
 [ 0.06782917 -0.03943818  0.12881066 -0.24950524  0.23112334  0.24432111
  -0.55209833 -0.00862108  0.07633565  0.69862777]]


TensorShape([Dimension(None), Dimension(128)])

In [207]:
model = K.Model(inputs=[input_x, input_xu, input_y, input_yu],
               outputs=[proximity])

In [208]:
i = 500
z = model.predict([X[:i], Xu[:i], Y[:i], Yu[:i]])


In [209]:
loss = K.losses.mean_squared_error(Yfake[:i], z)

L = K.backend.eval(loss)

In [210]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_y (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
input_yu (InputLayer)           (None, 128)          0                                            
__________________________________________________________________________________________________
input_x (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
input_xu (InputLayer)           (None, 128)          0                                            
__________________________________________________________________________________________________
embedding 

In [211]:
g = B.gradients(L, L_dense.get_weights())

g


[None, None]

In [212]:
opt = K.optimizers.Adam(lr=0.01)
model.compile(opt, loss='mean_squared_error')

In [214]:
model.fit([X, Xu, Y, Yu], [Yfake], batch_size=1024, epochs=1000000)


Epoch 1/1000000
157646/157646 [==============================] - 61s 386us/step - loss: 0.4093
Epoch 2/1000000
 10240/157646 [>.............................] - ETA: 57s - loss: 0.3843

KeyboardInterrupt: 

In [215]:
predictor = K.Model([input_x, input_xu], [dense])
predictor.compile(opt, loss="mean_squared_error")

In [216]:
from sklearn import preprocessing as pp
norm_emb_matrix =  pp.normalize(Lemb.get_weights()[0], axis=1)

In [217]:
len(X)

157646

In [219]:
i = 2008
gen = X[i].tolist()
genu = Xu[i].tolist()

tX = np.zeros((1, seqlen), dtype="int32")
tXu = np.zeros((1, seqlen), dtype="float32")
results = []

iterations = 128
for j in range(iterations):
    tX[0,:] = np.array(gen[-seqlen:], "int32")
    tXu[0,:] = np.array(genu[-seqlen:], "float32")
    #print(tX)
    xv, xuv = 0, 0.0
    z = predictor.predict([tX, tXu])
    if False: #z[0, -1, -1] > 0.9:
        xuv = 1.0
        results.append("<UNK>")
    else:
        we = np.transpose(pp.normalize(np.reshape(z[0, -1, :300], (1,300))))
#        print(np.linalg.norm(we))
#        scores = np.matmul(norm_emb_matrix, we)
#        print(scores)
#        print(np.min(scores))
#        break
        xv = np.argmax(np.matmul(norm_emb_matrix, we))
        if xv:
            results.append(words[xv])
        else:
            results.append("<EMPTY>")
    gen.append(xv)
    genu.append(xuv)
    print(results[-1])

that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that
that


In [66]:
print(np.min(z), np.max(z))

1.3547268e-16 0.93638355


In [92]:
from sklearn import preprocessing

In [93]:
norm_emb_matrix =  preprocessing.normalize(emb_matrix, axis=1)

In [94]:
zn = preprocessing.normalize(z[0,:], axis=1)


In [95]:
we = zn[-1, :300]

In [96]:
scores = np.matmul(norm_emb_matrix, we)

In [97]:
idx = np.argmax(scores)
print(idx)
word = words[idx]

23


In [98]:
word

'at'

In [84]:
words[2]

'.'